# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,4.28,76,30,11.89,RO,1701241878
1,1,whitehorse,60.7161,-135.0538,-13.16,92,15,1.79,CA,1701241667
2,2,albany,42.6001,-73.9662,-3.36,70,6,1.34,US,1701241889
3,3,waitangi,-43.9535,-176.5597,11.01,78,40,0.89,NZ,1701241681
4,4,longyearbyen,78.2186,15.6401,-0.09,86,75,4.12,SJ,1701241897


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) &
                                     (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
filtered_cities_df.dropna(how = "any")

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,port mathurin,-19.6833,63.4167,25.35,69,0,4.45,MU,1701241656
80,80,villamontes,-21.2500,-63.5000,23.29,77,0,1.04,BO,1701242033
306,306,vila do maio,15.1333,-23.2167,22.44,71,0,3.91,CV,1701242385
315,315,guanica,17.9716,-66.9080,25.54,66,0,2.20,PR,1701242419
424,424,mermoz boabab,14.7065,-17.4758,24.88,78,0,3.09,SN,1701242720
462,462,kidal,18.4411,1.4078,21.77,14,0,3.53,ML,1701242762


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,port mathurin,MU,-19.6833,63.4167,69,
80,villamontes,BO,-21.2500,-63.5000,77,
306,vila do maio,CV,15.1333,-23.2167,71,
315,guanica,PR,17.9716,-66.9080,66,
424,mermoz boabab,SN,14.7065,-17.4758,78,
462,kidal,ML,18.4411,1.4078,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
villamontes - nearest hotel: Hoterma
vila do maio - nearest hotel: Residencial Jardins De Maio
guanica - nearest hotel: Copamarina Beach Resort
mermoz boabab - nearest hotel: La maison blanche
kidal - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,port mathurin,MU,-19.6833,63.4167,69,Escale Vacances
80,villamontes,BO,-21.2500,-63.5000,77,Hoterma
306,vila do maio,CV,15.1333,-23.2167,71,Residencial Jardins De Maio
315,guanica,PR,17.9716,-66.9080,66,Copamarina Beach Resort
424,mermoz boabab,SN,14.7065,-17.4758,78,La maison blanche
462,kidal,ML,18.4411,1.4078,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)